# 单热编码

在第一部分，我们从零起步，创建了一个预测冰激淋销量的**多层神经网络**；在第二部分，我们亲手搭建了一个**神经网络训练框架**，支持全连接网络，解决了数值回归任务；在第三部分，我们进一步扩展了框架，使其具备了支持分类任务，处理多维度数据，和提取局部特征（CNN）的能力。

到目前为止，我们还停留在处理静态数据、“瞬时场景”的阶段。模型处理的每一组销量数据，识别的每一张图片，都是彼此独立的个体。

还有另一类数据，我们称之为**序列数据**（Sequence Data）。这一类数据，我们不能只看当前的一组数据，还要参考之前的所有数据。比如股价，我们不能只看今天的价格，还要参考之前很长时间的股价变化来做分析。

在所有序列数据中，最贴近人类智能、也最复杂的，莫过于语言。

这一部分，我们将继续扩充我们的神经网络训练框架，尝试处理文字信息。

---

我们需要面对的第一个问题是：如何数字化文字信息。

神经网络只能处理数字信息，我们需要一种方法将文字转换成数字。

一种最直接的方法是：把所有单词排列起来，分别给它们一个编号，称为**索引编码**（Index-Based Encoding）。

比如：假设排在前三位的单词分别是“你”、“我”、“他”，那么它们的编号就分别是 0、1、2。如果我们采用 GB-2312 汉字库，一共有 6763 个汉字，可以分别编号为 0 到 6762。

但是**索引编码**仍然不适用于神经网络，因为可能会让模型认为 1 代表的“我”大于 0 代表的“你”。

---

这时候，我们在上一部分的 MNIST 数据集中使用过的**单热编码**（One-Hot Encoding）正好可以派上用场。

**单热编码**的核心思想非常直观：它将每一个类别映射为一个向量，向量的长度和类别的总数相同。在这个向量中，只有一个位置的值是 1，而其余所有位置的值都是 0。

还以 GB-2312 汉字库为例。一共 6763 个汉字，所以每个汉字将被映射到一个长度为 6763 的向量。“你”的编码只有第 1 个位置是 1，“我”的编码只有第 2 个位置是 1。

|  词汇  |  索引编码  |          单热编码向量          |
|:----:|:------:|:------------------------:|
|  你   |   0    | [1, 0, 0, 0, 0, ... , 0] |
|  我   |   1    | [0, 1, 0, 0, 0, ... , 0] |
|  他   |   2    | [0, 0, 1, 0, 0, ... , 0] |

<div style="border-left: 4px solid #4CAF50; background:#f9f9f9; padding:10px; margin:10px 0;">
<strong>💡 提示：</strong> 单热编码虽然简单好用，但当词表达到几万甚至几十万时，向量会变得极其巨大，但是稀疏。
</div>

In [144]:
import csv
import re
from abc import abstractmethod, ABC
import numpy as np
from numpy.ma.core import argmax

np.random.seed(99)

## 基础架构

### 张量

In [145]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return np.prod(self.data.shape[1:])

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础数据集

In [146]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size, Tensor(self.labels).size

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

    def estimate(self, predictions):
        pass

## 数据

### IMDB 数据集

在这一部分，我们将使用来自 IMDB 的影评数据集。当然，为了加快模型训练速度，我们依旧制作了一个迷你版本：TinyIMDB。

IMDB 数据集的每个样本包括两个数据：

* **观众影评**：这是一段由观众书写的文字信息；
* **观众态度**：观众给出的评价（“喜欢”， 或者“讨厌”）。

---

在加载 IMDB 数据集时，我们将对数据进行预处理：

* **数据清洗**：数据清洗的目的在于去除观众影评中的 HTML 标签和标点符号；
* **构建词表**：统计数据集用到的所有单词，汇总成单词列表；
* **索引编码**：对单词列表进行索引编码；

数据加载以后，我们创建了几个列表属性：

* **词表**（vocabulary）：所有单词的汇总
* **索引表**：
    * **正向索引表**（word2index）：从单词到索引编码的对照表
    * **反向索引表**（index2word）：从索引编码到单词的对照表
* **词元表**（tokens）：将观众影评转换为索引编码

在 IMDB 数据集里，我们还提供了几个和编码有关的函数：

* **编码函数**（encode）：返回单词的索引编码；
* **解码函数**（decode）：返回索引编码对应的单词；
* **单热编码函数**（onehot）：返回索引编码的单热编码；
* **索引编码函数**（argmax）：返回单热编码对应的索引编码。

可以看出，我们没有进一步将**词元表**（tokens）转换成单热编码，不然我们需要巨量的内存空间来保存。我们提供了**单热编码函数**（onehot）在需要的时候可以实时将索引编码转换为单热编码。

In [147]:
class IMDBDataset(Dataset):

    def __init__(self, filename):
        self.filename = filename
        super().__init__()

    def load(self):
        self.reviews = []
        self.sentiments = []
        # 加载数据
        with open(self.filename, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)
            for _, row in enumerate(reader):
                self.reviews.append(row[0])
                self.sentiments.append(row[1])

        split_reviews = []
        for line in self.reviews:
            # 清洗、分词
            split_reviews.append(self.clean_text(line.lower()).split())

        # 构建词表
        self.vocabulary = set(word for line in split_reviews for word in line)
        # 索引编码
        self.word2index = {word: index for index, word in enumerate(self.vocabulary)}
        self.index2word = {index: word for index, word in enumerate(self.vocabulary)}
        # 将所有影评转换为索引编码
        self.tokens = [[self.word2index[word] for word in line if word in self.word2index] for line in split_reviews]

    @staticmethod
    def clean_text(text):
        # 去除 HTML 标签
        text = re.sub(r'<[^>]+>', '', text)
        # 去除标点符号
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return text

    def train(self):
        self.features = [list(set(index)) for index in self.tokens[:-10]]
        self.labels = [0 if index == "negative" else 1 for index in self.sentiments[:-10]]

    def eval(self):
        self.features = [list(set(index)) for index in self.tokens[-10:]]
        self.labels = [0 if index == "negative" else 1 for index in self.sentiments[-10:]]

    def encode(self, text):
        words = self.clean_text(text.lower()).split()
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        return " ".join([self.index2word[index] for index in tokens])

    def onehot(self, token):
        ebd = np.zeros(len(self.vocabulary))
        ebd[token] = 1
        return ebd

    @staticmethod
    def argmax(vector):
        return np.argmax(vector)

## 验证

### 测试

让我们加载数据集，看一看第一条数据（观众影评和观众态度）是什么。

In [148]:
# 加载数据集
dataset = IMDBDataset('tinyimdb.csv')
print(f'vocabulary count: {len(dataset.vocabulary)}')
print(f'review: "{dataset.reviews[0]}"')
print(f'tokens: {dataset.tokens[0]}')
print(f'sentiment: "{dataset.sentiments[0]}"')

vocabulary count: 86
review: "this movie was excellent. i enjoyed the plot and acting. the character was wonderful. recommend. screenplay actor actress by is a"
tokens: [36, 80, 83, 20, 64, 46, 61, 59, 50, 8, 61, 37, 83, 12, 23, 3, 14, 76, 5, 11, 38]
sentiment: "positive"


### 索引编码

In [149]:
message = 'i recommend this film'
print(f'message: "{message}"')

tokens = dataset.encode(message)
print(f'encode: {tokens}')
print(f'decode: "{dataset.decode(tokens)}"')

message: "i recommend this film"
encode: [64, 23, 36, 79]
decode: "i recommend this film"


### 单热编码

In [150]:
word = 'recommend'
print(f'word: "{word}"')

token = dataset.word2index[word]
print(f'token: {token}')

onehot = dataset.onehot(token)
print(f'onehot: {onehot}')
print(f'argmax: {dataset.argmax(onehot)}')

word: "recommend"
token: 23
onehot: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
argmax: 23
